<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Заказчик.** Страховая компания «Хоть потоп»

**Цель Заказчика.** Защитить данные клиентов компании таким образом, чтобы при преобразовании качество моделей машинного обучения не ухудшилось

**Цель исследования.** Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию

**Задачи:**

- Загрузить и изучить данные.
- Определить, изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу.
- Предложить алгоритм преобразования данных для решения задачи.
- Запрограммируйте предложенный алгоритм, применив матричные операции.


**Входные данные от Заказчика.** Файл в формате .csv с данными клиентов страховой компании

**Ожидаемый результат.** Предложен алгоритм, позволяющий защитить данные клиентов, и не ухудшающий качество моделей машинного обучения

## Загрузка данных

**Признаки**

- `Пол` - пол клиена
- `Возраст` - возраст клиента (полных лет)
- `Зарплата` - зарплата клиента за месяц
- `Члены семьи` - количество членов семьи у клиента

**Цеоевой признак**

- `Страховые выплаты` - количество страховых выплат клиенту за последние 5 лет


In [1]:
# Импотрт библиотек
import pandas as pd
import numpy as np
from numpy.random import RandomState
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # стандартизация
from sklearn.metrics import r2_score

In [2]:
# Введем константу
state = RandomState(12345)

In [3]:
# Прочитаем файл
df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
# Изучим состав данных методом info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
# Проверим наличие дубликатов и пропусков
print('Коичество явных дубликатов', df.duplicated().sum())
print('Количество пропусков:')
print(df.isna().sum())

Коичество явных дубликатов 153
Количество пропусков:
Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64


In [6]:
# Изучим состав данных
column_list = df.columns
for i in column_list:
    print(i)
    print(df[i].value_counts())

Пол
0    2505
1    2495
Name: Пол, dtype: int64
Возраст
19.0    223
25.0    214
31.0    212
26.0    211
27.0    209
22.0    209
32.0    206
28.0    204
29.0    203
30.0    202
23.0    202
21.0    200
20.0    195
36.0    193
33.0    191
24.0    182
35.0    179
34.0    177
37.0    147
39.0    141
38.0    139
41.0    129
18.0    117
40.0    114
42.0     93
43.0     77
44.0     74
45.0     73
46.0     60
47.0     47
49.0     37
50.0     27
48.0     26
52.0     22
51.0     21
53.0     11
55.0      9
54.0      7
56.0      5
59.0      3
60.0      2
58.0      2
57.0      2
65.0      1
61.0      1
62.0      1
Name: Возраст, dtype: int64
Зарплата
45800.0    29
37100.0    28
43200.0    27
41500.0    27
46800.0    26
           ..
14300.0     1
62600.0     1
7400.0      1
70000.0     1
15200.0     1
Name: Зарплата, Length: 524, dtype: int64
Члены семьи
1    1814
0    1513
2    1071
3     439
4     124
5      32
6       7
Name: Члены семьи, dtype: int64
Страховые выплаты
0    4436
1     423
2     1

In [7]:
# Изучим вероятность наличия неявных дубликатов
for i in column_list:
    print(i)
    print(df[i].sort_values().unique())


Пол
[0 1]
Возраст
[18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 65.]
Зарплата
[ 5300.  6000.  7400.  8900.  9800. 10000. 10600. 10800. 11000. 11200.
 11300. 12200. 12900. 13000. 13200. 13300. 13400. 13500. 13800. 13900.
 14100. 14300. 14400. 14500. 14600. 14700. 15000. 15100. 15200. 15600.
 15700. 15900. 16000. 16200. 16300. 16400. 16500. 16600. 16700. 17000.
 17100. 17300. 17400. 17500. 17600. 17700. 17800. 17900. 18100. 18200.
 18300. 18400. 18600. 18700. 18800. 18900. 19000. 19100. 19200. 19300.
 19400. 19600. 19700. 19900. 20000. 20100. 20200. 20300. 20400. 20500.
 20600. 20700. 20800. 20900. 21000. 21100. 21200. 21300. 21400. 21500.
 21600. 21700. 21800. 21900. 22000. 22100. 22200. 22300. 22500. 22600.
 22700. 22800. 22900. 23000. 23100. 23200. 23300. 23400. 23500. 23600.
 23700. 23800. 23900. 24000. 24100. 24200. 24300. 24400. 24500. 24600.
 247

In [8]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


**Выводы**

Файл содержит 5000 строк и 5 столбцов. Тип данных корректный и отражает их смысл. Пропусков данных нет. Присутствуют явные дубли. Неявных дублей не обнаружено.

Есть перекос целевого показателя в сторону клиентов, у которых нет страховых выплат.

**Требует изменения:**

- Некорректное название столбцов. Требуется привести в соответствие со стандартами
- Наличие дублей. Необходимо удалить

In [9]:
# Удалим явные дубликаты
df = df.drop_duplicates().reset_index(drop=True)
print('Коичество явных дубликатов', df.duplicated().sum())

Коичество явных дубликатов 0


In [10]:
# Ихменним назвнаия стобцов
df = df.rename(columns={
    'Пол': 'gender', 
    'Возраст': 'age', 
    'Зарплата': 'salary', 
    'Члены семьи': 'family_members', 
    'Страховые выплаты': 'insurance_payments'
})
df.head()

,gender,age,salary,family_members,insurance_payments
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [11]:
# Разделим таблицу на общие и целевой признаки и преобразуем в матрицу
features = df.drop('insurance_payments', axis=1)
target = df['insurance_payments']

X = np.array(features)
y = np.array(target)

## Умножение матриц

**Определить, изменяется ли качество линейной регрессии при умножении признаков на обратимую матрицу:**

- При изменении привести примеры матриц
- В случае отсутствия изменений, указать, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

In [12]:
# Выделим обучающюую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=state)

# Создадим модель линейной регрессии и проерим ее качество на тестовой выборке
model = LinearRegression()
model.fit(X_train, y_train)
predict = model.predict(X_test)
result = r2_score(y_test, predict)
print(f'Параметры модели: {model.coef_}')
print(f'На исходных данных качество прогноза по показателю R2 составило {result:.4f}')

Параметры модели: [ 1.45766002e-02  3.64782926e-02  1.79477716e-07 -1.23345013e-02]
На исходных данных качество прогноза по показателю R2 составило 0.4231


In [13]:
# Проверим расчет параметров модели
X_control = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
w_comtrol = (np.linalg.inv(X_control.T @ X_control) @ X_control.T).dot(y_train)
w_comtrol = w_comtrol[1:]
w_comtrol

array([ 1.45766002e-02,  3.64782926e-02,  1.79477716e-07, -1.23345013e-02])

Параметры модели Линейной регрессии полностью совпадают с рассчитанными вручную

In [14]:
# Создадим обратимую матрицу и скалярно умножим на нее исходную
P = np.random.normal(size=(4, 4))
print('Обратимая матрица')
print(P)
print()

Xp = X @ P
print('Новая матрица')
print(Xp)

Обратимая матрица
[[ 0.35581242  0.04576575  0.16374369  0.54734676]
 [ 0.4729074  -0.62050005 -1.60407276  0.86614821]
 [-0.90411172 -0.08759324 -1.27179441 -1.72259694]
 [ 2.47893575 -0.12432429 -0.38075829  0.55610072]]

Новая матрица
[[-44821.71713537  -4370.14389163 -63146.98668014 -85404.19289512]
 [-34332.01251137  -3357.21054399 -48402.35564424 -65418.28495111]
 [-18972.63171007  -1857.4525953  -26754.20069739 -36149.41752451]
 ...
 [-30634.9711347   -2982.06957237 -43146.67343409 -58377.60123382]
 [-29546.25651316  -2878.27723994 -41623.94530352 -56307.64915718]
 [-36690.85949242  -3573.7382079  -51679.98405396 -69912.08032646]]


In [15]:
# Выделим обучающюую и тестовую выборки для новой матрицы
Xp_train, Xp_test, yp_train, yp_test = train_test_split(Xp, y, test_size=.25, random_state=state)

# Создадим модель линейной регрессии и проерим ее качество на тестовой выборке для новой мартицы
model_p = LinearRegression()
model_p.fit(Xp_train, yp_train)
predict_p = model_p.predict(Xp_test)
result_p = r2_score(yp_test, predict_p)
print(f'Параметры модели: {model_p.coef_}')
print(f'На новой матрице качество прогноза по показателю R2 составило {result:.4f}')

Параметры модели: [-0.01468893  0.04747359 -0.02999215  0.02743857]
На новой матрице качество прогноза по показателю R2 составило 0.4231


In [16]:
# Проверим расчет параметров модели после умножения на обратимую марицу
Xp_control = np.concatenate((np.ones((Xp_train.shape[0], 1)), Xp_train), axis=1)
wp_comtrol = (np.linalg.inv(Xp_control.T @ Xp_control) @ Xp_control.T).dot(yp_train)
wp_comtrol = wp_comtrol[1:]
wp_comtrol

array([-0.01468912,  0.0474784 , -0.02999357,  0.02743948])

Параметры новой модели Линейной регрессии полностью совпадают с рассчитанными вручную

Значение метрики R2 у исходной модели полностью совпадает со значением метрики R2 у изменённой модели. Таким образом, **качество линейной регрессии не меняется** при умножении признаков на обратимую матрицу.

**Ответ:** 
При умножении признаков на обратимую матрицу качество линейной регрессии **не меняется**


**Обоснование:** 

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки (*обратимая матрица*)

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $w_p$ — вектор весов НОВОЙ линейной регрессии (после умножения на обратимую матрицу)

Свойства матриц:

1. Матрица, умноженная на обратную, дает единичную матрицу:

$$
(A)^{-1} A = E
$$

2. Скалярное умножение матриц с последующим транспонированием равно скалярному умножению уже транспонированных матриц:

$$
(A B)^T = B^T A^T
$$

3. Перевод в обратную скалярно умноженных матриц равно скалярному умножению матриц, уже переведенных в обратные:

$$
(A B)^{-1} = (B)^{-1} (A)^{-1}
$$

4. Свойство ассоциативности:

$$
(A B) C = A (B C)
$$

Предсказания:

$$
a = X w
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

*Следовательно:*

$$
a = X (X^T X)^{-1} X^T y
$$

Новая матрица (после умножения на обратимую матрицу):

$$
A = XP
$$

Предсказания:

$$
a_p = A w_p
$$

Формула обучения:

$$
w_p = (A^T A)^{-1} A^T y
$$

*Следовательно:*

$$
a_p = A (A^T A)^{-1} A^T y
$$

*Т.к.* $A = XP $, *то:*

$$
a_p = XP ((XP)^T XP)^{-1} (XP)^T y
$$

*Т.к.* $(A B)^T = B^T A^T $, а $(A B) C = A (B C) $ *то:*

$$
a_p = XP ((XP)^T XP)^{-1} (XP)^T y = XP (P^T X^T X P)^{-1} (XP)^T y
$$

$$
a_p = XP (P^T X^T X P)^{-1} (XP)^T y = XP ((P^T X^T X) P)^{-1} (XP)^T y
$$

*Т.к.* $(A B)^{-1} = (B)^{-1} (A)^{-1} $, *а матрица* $P$ *обратимая, то:*

$$
a_p = XP ((P^T X^T X) P)^{-1} (XP)^T y = XP (P)^{-1} (P^T (X^T X))^{-1}  (XP)^T y
$$

$$
a_p = XP (P)^{-1} (P^T (X^T X))^{-1} (XP)^T y = XP (P)^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

*Т.к.* $(A)^{-1} A = E$, a $A E = A $ *то:*

$$
a_p = XP (P)^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = XP E (X^T X)^{-1} E X^T y
$$

$$
a_p = XP E (X^T X)^{-1} E X^T y = XP (X^T X)^{-1} X^T y = a
$$


## Алгоритм преобразования

Для решения задачи защиты данных клиентов, будет произведено умножение матрицы признаков на сгенерированную случайным образом обратимую матрицу.

**Алгоритм**

1. Формирование обратимой матрицы $P$
2. Проверка сформированной матрицы на обратимость через вычисление определителя
3. Получение преобразованной матрицы признаков $A = X P$.
4. Применение метода линейной регрессии на преобразованной матрице признаков $A$

**Обоснование**

Для преобразования матрицы признаков, обратимая матрица $P$ должна иметь такую размерность $n * n$, где $n$ соответствует числу признаков исходной матрице $X$. 

Проверить обратимость матрицы $P$ можно сравнив ее определитель с нулем. Если определитель будет отличен от нуля, то матрица обратимая

Пример:

$$
X = \begin{pmatrix}
  1 & 2\\
  3 & 4\\
  5 & 6
\end{pmatrix}
$$

$$
P = \begin{pmatrix}
  7 & 8\\
  9 & 0
\end{pmatrix}
$$

$$
A = X P
$$

$$
A = \begin{pmatrix}
  25 & 8\\
  57 & 24\\
  89 & 40
\end{pmatrix}
$$

Размерность проборазовнанной мартицв сохранена

Добавим нулевой столбец и для ввода данные в линейную регрессию

$$
A = \begin{pmatrix}
  1 & 25 & 8\\
  1 & 57 & 24\\
  1 & 89 & 40
\end{pmatrix}
$$


## Проверка алгоритма

- Запрограммировать предложенный алгоритм, применив матричные операции.
- Проверите, что качество линейной регрессии из sklearn не отличается до и после преобразования. 
- Применить метрику R2.

In [17]:
# Стандартизируем признаки
num = ['age', 'salary']
features_st = features
scaler = StandardScaler()
scaler.fit(features_st[num])
features_st[num] = scaler.transform(features_st[num])
display(features_st.head())
X_st = np.array(features_st)

,gender,age,salary,family_members
0,1,1.175436,0.973151,1
1,0,1.764564,-0.190115,1
2,0,-0.238471,-1.894901,0
3,0,-1.181076,0.180927,2
4,1,-0.356297,-1.383465,0


In [18]:
# Выделим обучающюую и тестовую выборки
X_st_train, X_st_test, y_st_train, y_st_test = train_test_split(X_st, y, 
                                                                test_size=.25, 
                                                                random_state=state)
print('Размер обучающей выборки', X_st_train.shape)
print('Размер тестовой выборки', X_st_test.shape)

Размер обучающей выборки (3635, 4)
Размер тестовой выборки (1212, 4)


Создадим модель линейной регрессии на **исходных** данных

In [19]:
# Создадим и обучим модель линейной регрессии
model = LinearRegression()
model.fit(X_st_train, y_st_train)
predict_st = model.predict(X_st_test)

# Проверим качество модели на тестово выбоке метрикой R2
result_st = r2_score(y_st_test, predict_st)
print(f'На исходных данных точность прогноза по показателю R2 составило {result_st:.4f}')

На исходных данных точность прогноза по показателю R2 составило 0.4331


Создадим модель линейной регрессии на **пробразованных** данных

In [20]:
#Создадим обратимую матрицу
p_size = X.shape[1]
P = np.random.normal(size=(p_size, p_size))
print(P)

#Проверим обратимость созданной матрицы
p_det = np.linalg.det(P)
if p_det != 0:
    print(f'Определитель отличен от нуля, равен {p_det:.4f}, значит матрица обратимая')
else:
    print('Определитель равен нулю, значит матрица НЕ обратимая')

[[ 0.27807156  0.5046123   0.75907584 -1.54015799]
 [ 0.60816445  0.88911154 -0.9835098   1.12111215]
 [ 0.93733172  0.13570881 -1.89220417 -0.97423455]
 [-0.11932174  0.12668143 -0.78442387 -1.70629881]]
Определитель отличен от нуля, равен 3.7193, значит матрица обратимая


In [21]:
# Преобразуем исходные признаки
A_st_train = X_st_train @ P
A_st_test = X_st_test @ P

In [22]:
# Создадим и обучим модель линейной регрессии
model_a = LinearRegression()
model_a.fit(A_st_train, y_st_train)
predict_a = model_a.predict(A_st_test)

# Проверим качество модели на тестово выбоке метрикой R2
result_a = r2_score(y_st_test, predict_a)
print(f'На преобразованных данных точность прогноза по показателю R2 составила {result_a:.4f}')

На преобразованных данных точность прогноза по показателю R2 составила 0.4331


**Выводы**

В результате выполнения проект доказано, что умножение матрицы исходных признаков на обратимую матрицу не меняет качество прогноза методом линейной регрессии. При этом, происходит значительное изменение исходной информации, что позволяет срыть и обезопасить личные данные клиентов.

При изменение исходных данные (без стандартизации) показатель `R2` не меняется и составляет `0.4231`, как на первичных признаках, так и на измененных, путем умножения на оборотную матрицу. 

При использовании стандартизации данных, показатель `R2` также не меняется, но составляет `0.4331`, как на первичных, так и на преобразованных признаках.

**Таким образом, предложенный алгоритм, позволяет защитить данные клиентов, не ухудшая качество моделей машинного обучения**